In [91]:
'''
Title:       Optimale Steuerung und Regelung:
Subttitle:   1. Aufgabe
Author:      Stefan Kaufmann
MaNr.        51867606
Date:        01.04.2023
'''

'\nTitle:       Optimale Steuerung und Regelung:\nSubttitle:   1. Aufgabe\nAuthor:      Stefan Kaufmann\nMaNr.        51867606\nDate:        01.04.2023\n'

# Optimale Steuerung und Regelung
## 1. Übung
### Stefan Kaufmann - 51867606

In [270]:
import numpy as np
import scipy as sp
import scipy.linalg as la
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.optimize import minimize

mathematisches Pendel einer oszillierenden Punktmasse mit PID Regler    
$m \ddot{y}(t) + \omega^{2}y(t) = u(t)  \hspace{2cm} y(0) = 1, \hspace{1cm} \dot{y} = 0 \\
u(t) = K_{p}y(t) + K_{D}\dot{y}(t) + K_{I} \int_{0}^{t}y(\tau) d\tau 
$  
mit dem Kostenfunktional    
$
F(k,y(k;t)) = \frac{1}{2} \int_{0}^{\inf} (x^{T}(t)Qx(t) + ru^{2})dt
$


## a) Überführung in ein satisches Problem

$ x = 
\begin{bmatrix}
y & \dot{y} & \int_{0}^{t}y(\tau) d\tau 
\end{bmatrix} ^{T}
$

$
\dot{x} = 
\begin{bmatrix}
0 & 1 & 0  \\
\frac{-\omega^{2}}{m} & 0 & 0 \\
1 & 0 & 0
\end{bmatrix} x
+
\begin{bmatrix}
0 \\ \frac{1}{m} \\ 0
\end{bmatrix} u
$     
$
y = 
\begin{bmatrix}
1 & 0 & 0
\end{bmatrix} + 0u
$   
$
F(k,y(k;t)) = \frac{1}{2} \sum_{k=0}^{N} (x_{k}^{T}(t)Qx_{k}(t) + ru_{k}^{2})dt \\
mit \hspace{1cm}  u = kx_{k}  \hspace{1cm}  x_{k+1} = Ax_{k} + Bu_{k}  = (A+Bk)x_{k}
$

In [152]:
# Parameter
m  = 1
w  = 2
Q = la.block_diag(3,4,5)
R = 6

In [158]:
# State Space System
A = np.array([[0,       1, 0],
               [-w**2/m, 0, 0],
               [1,       0, 0]])

B = np.array([[0],[1/m],[0]])

C = np.array([[1,0,0]])
D = 0

In [193]:
def runge_kutta_k4(f,x,u,h=0.1):
    """ Runge Kutta Verfahren mit variabler Schrittweite 
        Params
         --------
        f:             Funktion welche integriert werden soll
        x:             Startvektor x0 = [x1 x2 x3 x4 x5 .... xN]
        u:             Eingangsvektor u = [u1 u2 u3 .... uM]
        h:             Schrittweite 
        t:             Hilfsvariable --> wird benötigt da f eine Funktion aus f(t,x,u) ist


        Returns
        --------
        x + dx:       neuer Zustandsvektor  [x1 x2 .... xN]   
                
    """
    #RK4 integration with zero-order hold on u
        
    f1 = f(x, u)
    f2 = f(x + 1/2*h*f1, u)
    f3 = f(x + 1/2*h*f2, u)
    f4 = f(x + h*f3, u)

    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)

In [285]:
global x0,N,nx, k0
f_dynamic = lambda x,u: A@x+B@u
k0 = np.array([[-2,-2,-2]])         # Anfangsschätzung
N = 1000                           # Anzahl von Zeitschritten N--> unendlich
nx = 3                             # Anzahl von Zuständen
x0 = np.array([[1],[0],[0]])       # Startzustand --> zu stabilisiern
dt = 0.01

Ad,Bd,Cd,Dd,Ta = sig.cont2discrete((A,B,C,D),dt, method='foh')  # Discretisierung damit dt --> gegen 0 geht  (= analytische Lösung)

def rollout(k,x):
    u     = k@x    
    dx    = Ad@x + Bd@u    
    #dx    =A@x +B*u
    #dx = runge_kutta_k4(f_dynamic,x,u)
    return dx , u

def cost(k):  
    k = np.reshape(k, (1, 3))   
    x_new, u   = rollout(k,x0)  

    cost_      = (x0.T@Q@x0 + R*u**2*0)/2
   
    for i in range(1,N-2): 
        # laufende Kosten
        x_new, u = rollout(k,x_new)       
        
        cost_ += x_new.T@Q@x_new/2
        cost_ += u**2*R/2
   
    return cost_  

In [199]:
# Lösung der algebraischen Ricatti Gleichung
P = la.solve_continuous_are(A,B,Q,R)
K = -B.T@P/R
print('analytische Lösung',K)
Pd = la.solve_discrete_are(Ad,Bd,Q,R)
Kd = -Bd.T@Pd/R
print('zeitdiskrete Lösung',Kd)

analytische Lösung [[-0.30926715 -1.13366704 -0.91287093]]
zeitdiskrete Lösung [[-0.3473642  -1.13682087 -0.91805992]]


In [286]:
# Kontrolle der Kostenfunktion
res = minimize(cost, k0, method="nelder-mead", options={"disp": True})
print(res)

Optimization terminated successfully.
         Current function value: 1193.681374
         Iterations: 83
         Function evaluations: 149
 final_simplex: (array([[-0.29879199, -1.10662862, -0.84084109],
       [-0.29881505, -1.10665951, -0.84093693],
       [-0.29877403, -1.10663759, -0.8408599 ],
       [-0.29877662, -1.10663114, -0.84088082]]), array([1193.68137397, 1193.68137399, 1193.68137404, 1193.68137406]))
           fun: 1193.6813739688723
       message: 'Optimization terminated successfully.'
          nfev: 149
           nit: 83
        status: 0
       success: True
             x: array([-0.29879199, -1.10662862, -0.84084109])


In [217]:
# Alternative mit Scipy integrate
from scipy.integrate import quad
from scipy.linalg import expm

def integrand(t,k):
    k = np.reshape(k, (3, 1))
    x = expm((A + B@k.T)*t)@x0              # Lösung der Matrixespotenitlagleichung
    u = k.T@x                             # Eingang
    #print((x@Q@x + R*u**2)*0.5)
    return (x.T@Q@x + R*u**2)*0.5


cost3 = lambda k: quad(integrand, 0, np.inf, args=(k,))[0]

res3 = minimize(cost3, k0, method="nelder-mead", options={"disp": True})
print(res3)


Optimization terminated successfully.
         Current function value: 11.917210
         Iterations: 84
         Function evaluations: 155
 final_simplex: (array([[-0.30923162, -1.13364782, -0.91285598],
       [-0.30927895, -1.13363177, -0.91280413],
       [-0.30927403, -1.13363048, -0.91276123],
       [-0.30931058, -1.13373422, -0.91295376]]), array([11.91720968, 11.91720968, 11.91720969, 11.91720969]))
           fun: 11.917209681931078
       message: 'Optimization terminated successfully.'
          nfev: 155
           nit: 84
        status: 0
       success: True
             x: array([-0.30923162, -1.13364782, -0.91285598])


In [284]:
# Genenüberstellung
print('analytisch', K )
print('Zeitsikret endlich', res.x)
print('scipy integrade', res3.x)

analytisch [[-0.30926715 -1.13366704 -0.91287093]]
Zeitsikret endlich [-0.29879199 -1.10662862 -0.84084109]
scipy integrade [-0.30923162 -1.13364782 -0.91285598]


## b) Gradient des Kostenfunktionals


In [344]:
def dcost(k):    
    gradient = np.zeros((nx,1),dtype=float)
    gradient += x0@k@x0*R
    xnew,u   = rollout(k,x0)
    
    for i in range(0,N):                         
        #gradient += (xnew.T@Q).T  
        gradient  += (Ad+Bd@k).T@xnew     
        gradient  += xnew@k@xnew*R

        xnew,u  = rollout(k,xnew)
    
    return gradient

In [316]:
from scipy.optimize import approx_fprime

def dcost3(k):
    # Nummerisch Ableiten 
    return approx_fprime(k, cost3, 1e-8)

In [345]:

print(dcost(k0).T)
print(dcost3([-2,-2,-2]))

[[    6.00875618 -1307.43072707   399.48219725]]
[-2.59999988  0.1750001   0.41250008]


In [346]:
# Kontrolle des Gradienten
res = minimize(cost, k0, method="BFGS",jac=dcost3, options={"disp": True})
print(res)

/tmp/ipykernel_3797/1595469226.py:10: RuntimeWarning: overflow encountered in matmul
  return (x.T@Q@x + R*u**2)*0.5
/tmp/ipykernel_3797/1595469226.py:10: RuntimeWarning: overflow encountered in square
  return (x.T@Q@x + R*u**2)*0.5
/home/stefan/.local/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


         Current function value: 1193.794703
         Iterations: 10
         Function evaluations: 72
         Gradient evaluations: 57
      fun: 1193.7947032458635
 hess_inv: array([[0.40610388, 0.21089909, 0.46538967],
       [0.21089909, 0.28966817, 0.63122082],
       [0.46538967, 0.63122082, 1.91595445]])
      jac: array([0.00429239, 0.00915374, 0.00060432])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 72
      nit: 10
     njev: 57
   status: 2
  success: False
        x: array([-0.30559221, -1.12992055, -0.90451541])


In [347]:
res = minimize(cost3, k0, method="BFGS",jac=dcost3, options={"disp": True})
print(res)

Optimization terminated successfully.
         Current function value: 11.917210
         Iterations: 13
         Function evaluations: 17
         Gradient evaluations: 17
      fun: 11.917209679572888
 hess_inv: array([[0.38209116, 0.20709765, 0.44760797],
       [0.20709765, 0.28954696, 0.62664103],
       [0.44760797, 0.62664103, 1.86617853]])
      jac: array([-7.10542736e-07,  0.00000000e+00, -3.55271366e-07])
  message: 'Optimization terminated successfully.'
     nfev: 17
      nit: 13
     njev: 17
   status: 0
  success: True
        x: array([-0.30926735, -1.13366716, -0.9128712 ])


## c) Gradientenverfahren nach [Gra19, Abschnitt 3.3.2]
Die Suchrichtung ergibt sich zu: $ \hspace{0.5cm} s^{k}= -\nabla f(x^{k}) $
## d) Schrittweitensteuerung mittels Backtracking Verfahren
![amerigo](./img/amerigo.png)



In [362]:
def backtracking(x,f,s,rho=0.5,c1=1e-7):

    alpha = 1   

    while f(x +alpha*s) >= f(x) + c1*alpha*s@s or np.isnan(f(x+alpha*s)):
        alpha *= rho   # Reduzierung der Schrittweite     

    return alpha

In [366]:
def gradienten_step(x,f,df,fix=False):
        
    # Gradientenrichtung --> ohne Regularisierung
    s = -df(x)               
    
    # Backtracking    
    alpha = backtracking(x,f,s)
    # Fixed Step size
    if fix == True:
        alpha = 0.1
    
    

    return x + alpha*s 


In [364]:
def linesearch(x0,f,df,Nmax=20,tol_x=1e-20, tol_f =1e-10, fix_stepsize=False ):
        
    x_alt = x0.copy()

    for k in range(Nmax):
              
        x = gradienten_step(x_alt,f,df,fix_stepsize)        
        # Abbruchkriterium
        if np.linalg.norm(df(x)) < tol_f: #np.linalg.norm(x_alt-x) < tol_x: #or np.linalg.norm(df(x_alt)-df(x)) < tol_f:            
                return x,k
        print(k, 'xopt= ',x)  
        x_alt = x.copy()
    return x,k
        

In [367]:
k0 = np.array([-2, -2, -2])
xopt, iter = linesearch(k0,cost3,dcost3, fix_stepsize = True)

print('xopt', xopt)
print('Iterationen', iter)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
xopt [-0.57888649 -1.50397805 -2.08617352]
Iterationen 19
